In [1]:
import sys, pathlib, os
sys.path.append(str(pathlib.Path(os.path.abspath("")) / ".."))
import numpy as np
import NuLattice.lattice as lat
import NuLattice.references as ref
import NuLattice.CCM.coupled_cluster as ccm

In [2]:
# Initialize lattice
thisL = 4   #L*L*L lattice
a_lat = 2.0 # lattice spacing in fm
phys_unit = lat.phys_unit(a_lat)

my_basis = lat.get_sp_basis(thisL)
nstat =  len(my_basis)
print("number of single-particle states =", nstat)
lattice = lat.get_lattice(thisL)
nsite = len(lattice)
print("number of lattice sites =", nsite)

number of single-particle states = 256
number of lattice sites = 64


In [3]:
# Compute operators for kinetic energy, two-body contacts, and three-body contact

vT1=-8.0  #S-wave isospin-triplet contact
vS1=-8.0  #S-wave spin-triplet contact
cE = 5.5  #three-body contact


myTkin=lat.Tkin(lattice, thisL)
print("number of matrix elements from kinetic energy", len(myTkin))

mycontact=lat.contacts(vT1, vS1, lattice, thisL)
print("number of matrix elements from two-body contacts", len(mycontact))

my3body=lat.NNNcontact(cE, lattice, thisL)
print("number of matrix elements from three-body contacts", len(my3body))

number of matrix elements from kinetic energy 1792
number of matrix elements from two-body contacts 512
number of matrix elements from three-body contacts 256


In [4]:
# reference state
ref_state = ref.ref_16O_gs

#Choose whether or not to have v_pppp and v_ppph sparse or not
sparse = True
# make normal-ordered Hamiltonian
refEn, fock_mats, two_body_int = ccm.get_norm_ordered_ham(
    thisL, ref_state, myTkin, mycontact, my3body, sparse=sparse, NO2B=True)


print("energy of reference:", refEn*phys_unit)
#adding a delta to shift the first iteration, avoiding divide by 0 errors
delta = 0

#Whether or not to print out each iteration of the coupled cluster calculation
verbose = True

#solving the coupled cluster equations until we get to a relative error of 1e-8
corrEn, t1, t2 = ccm.ccsd_solver(fock_mats, two_body_int, eps = 1.e-8, maxSteps = 100, 
                                 max_diis = 10, delta = delta, mixing = 0.5,
                                 sparse=sparse, verbose=verbose, ccs=False)

#converting the energy from lattice units to MeV
gsEn = (corrEn + refEn) * phys_unit

print(f'The ground state energy on a L={thisL} size lattice is {gsEn} MeV')


energy of reference: -41.47103264313923
Step 0: -9.6
Step 1: -9.195597181625239 difference = 0.04397787445309631
Step 2: -9.305347477800229 difference = 0.011794325406635447
Step 3: -9.492656874239668 difference = 0.019732030654952062
Step 4: -9.659886351535555 difference = 0.01731174376283464
Step 5: -9.792027535615683 difference = 0.013494772517693841
Step 6: -9.89332876531433 difference = 0.010239347352309337
Step 7: -9.9709364165197 difference = 0.007783386430665803
Step 8: -10.031027060844602 difference = 0.005990477740755099
Step 9: -10.0782553078891 difference = 0.004686153069324194
Step 10: -10.115981532118836 difference = 0.0037293686341709187
Step 11: -10.342805026911908 difference = 0.021930558896051763
Step 12: -10.343359538309215 difference = 5.361037632444334e-05
Step 13: -10.343636141751391 difference = 2.6741412631408765e-05
Step 14: -10.343762507118866 difference = 1.2216576645859939e-05
Step 15: -10.343819175970738 difference = 5.478523058874886e-06
Step 16: -10.34385